## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2) #These are coordinates of chessboard corners in the real frame

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('../camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None) #These are the pixel coordinates of the corners

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        #cv2.imshow('img',img)
        #cv2.waitKey(500)

cv2.destroyAllWindows()

image_size=img.shape

#The calibration and distortion matrices
ret,cameraMatrix,distCoeffs, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, (img.shape[::-1][2],img.shape[::-1][1]), None, None)

#Correct for distortions
#for fname in images:
    #src=cv2.imread(fname)
    #src=cv2.undistort(src, cameraMatrix, distCoeffs)
    #cv2.imshow('img',src)
    #cv2.waitKey(500)
    
cv2.destroyAllWindows()

## Create Black and white image

In [2]:
def get_abs_gradient(image,dx,dy):
    img_grad=cv2.Sobel(image,cv2.CV_64F,dx,dy,ksize=-1)
    img_grad=np.abs(img_grad)
    return np.uint8(img_grad)

In [48]:
def get_binary_image(image,cameraMatrix,distCoeffs):
    image=cv2.undistort(image,cameraMatrix,distCoeffs)
    image=cv2.GaussianBlur(image,(9,9),0)
    #image=cv2.Sobel(image,cv2.CV_64F,1,0,ksize=-1)
    
    hls_img=cv2.cvtColor(image,cv2.COLOR_BGR2HLS)
    
    #hls_img=cv2.GaussianBlur(hls_img,(9,9),0)
    hls_img_grad=get_abs_gradient(hls_img,1,0)
    s_img=hls_img[:,:,2]
    s_img=cv2.adaptiveThreshold(s_img,200,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv2.THRESH_BINARY,3,-1) #This is to detect yellow
    
    binary_img=cv2.adaptiveThreshold(hls_img[:,:,2],200,cv2.ADAPTIVE_THRESH_MEAN_C,\
            cv2.THRESH_BINARY,5,-1) #This is to detect yellow lines
    

    ret,white_threshold=cv2.threshold(hls_img[:,:,1],150,100,type=0)
    ret,grad_threshold=cv2.threshold(hls_img_grad[:,:,1],150,100,type=0)
    
    ret,sat_grad_threshold=cv2.threshold(hls_img_grad[:,:,2],150,100,type=0)
    
    lum_cutoff=cv2.bitwise_and(grad_threshold,white_threshold)
    gradient_image=cv2.bitwise_or(lum_cutoff,sat_grad_threshold)
    
    
    return gradient_image
    return cv2.bitwise_or(gradient_image,s_img)

    

In [4]:
def get_binary_image2(image,cameraMatrix,distCoeffs):
    image=cv2.undistort(image,cameraMatrix,distCoeffs)
    image=cv2.GaussianBlur(image,(9,9),10)
    #image=cv2.Sobel(image,cv2.CV_64F,1,0,ksize=-1)
    
    hls_img=cv2.cvtColor(image,cv2.COLOR_BGR2HLS)
    
    #hls_img=cv2.GaussianBlur(hls_img,(9,9),0)
    hls_img_grad=cv2.Sobel(hls_img,cv2.CV_64F,1,0,ksize=-1)
    s_img=hls_img[:,:,2]
    s_img=cv2.adaptiveThreshold(s_img,200,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
           cv2.THRESH_BINARY,5,0)
    
    #s_img=cv2.adaptiveThreshold(s_img,200,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
    #       cv2.THRESH_BINARY,3,-5)
    
    yellow_hsv_low  = np.array([ 120, 0, 12])
    yellow_hsv_high = np.array([ 180, 255, 30])
    hue_img=hls_img[:,:,2]
    hue_img=cv2.inRange(hls_img,yellow_hsv_low,yellow_hsv_high)
    
    
    
    lab_img=cv2.cvtColor(image,cv2.COLOR_BGR2LAB) #The LAB channel is useless for yellow detection in this case
    binary_img=cv2.adaptiveThreshold(hls_img[:,:,1],200,cv2.ADAPTIVE_THRESH_MEAN_C,\
            cv2.THRESH_BINARY,5,-1)
    
    sat_image_blurred=cv2.GaussianBlur(hls_img[:,:,2],(11,11),10)
    lum_image_blurred=cv2.GaussianBlur(hls_img[:,:,1],(11,11),10)
    for i in range(0):
        sat_image_blurred=cv2.GaussianBlur(sat_image_blurred,(19,19),10)
    
    sobel_x=cv2.Sobel(sat_image_blurred,cv2.CV_64F,1,0,ksize=-1)
    sobel_x=np.abs(sobel_x)
    sobel_x=np.uint8(sobel_x)
    
    sobel_l=cv2.Sobel(lum_image_blurred,cv2.CV_64F,1,0,ksize=3)
    sobel_l=np.abs(sobel_l)
    sobel_l=np.uint8(sobel_l)
    
    ret,sobel_x=cv2.threshold(sobel_x,60,200,cv2.THRESH_BINARY)
    #ret,sobel_l=cv2.threshold(sobel_l,60,200,cv2.THRESH_BINARY)
    #sobel_x=cv2.adaptiveThreshold(sobel_x,200,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
    #       cv2.THRESH_BINARY,3,-5)
    
    
    scaled_sobel=np.uint8(255*sobel_x/np.max(sobel_x))
    
    laplacian=cv2.Laplacian(hls_img[:,:,2],cv2.CV_64F)
    laplacian=np.abs(laplacian)
    laplacian=np.uint8(laplacian)
    
    canny_img=cv2.Canny(sobel_x,60,190,-1)
    
    return s_img
    return cv2.bitwise_or(binary_img,sobel_l,sobel_x)

In [5]:
challenge_image=cv2.imread('../challenging_image.png')
challenge_binary=get_binary_image(challenge_image,cameraMatrix,distCoeffs)

challenge_binary=np.uint8(challenge_binary)
cv2.imshow('challenge_binary',challenge_binary)

#np.unique(challenge_binary)
challenge_binary.shape

(661, 873)

In [6]:
test_images_directory = glob.glob('../test_images/test*.jpg')
save_images_directory='../output_images/'
test_images=[cv2.imread(img_name) for img_name in test_images_directory]

for image_no,image in enumerate(test_images):
    
    binary_image=get_binary_image(image,cameraMatrix,distCoeffs)
    cv2.imwrite(save_images_directory+'test'+str(image_no+1)+'.jpg',binary_image)
    #cv2.imwrite(save_images_directory+'canny/''test'+str(image_no+1)+'.jpg',canny_img)
    #cv2.imshow('image',canny_img)
    cv2.waitKey(20)

cv2.destroyAllWindows()

In [7]:
class ImageWarper:
    def __init__(self):
        self.calc_warp_points(720,1280)
        self.get_transforms()
        
    def calc_warp_points(self,img_height,img_width,x_center_adj=0):

        # calculator the vertices of the region of interest
        imshape = (img_height, img_width)
        xcenter=imshape[1]/2+x_center_adj
    #     xfd=55
    #     yf=450
    #     xoffset=100
        xfd=54
        yf=450
        xoffset=120

        src = np.float32(
            [(xoffset,imshape[0]),
             (xcenter-xfd, yf), 
             (xcenter+xfd,yf), 
             (imshape[1]-xoffset,imshape[0])])

        dst = np.float32(
            [(xoffset,imshape[1]),
             (xoffset,0),
             (imshape[0]-xoffset, 0),
            (imshape[0]-xoffset,imshape[1])])

        self.src=src
        self.dst=dst

    def get_transforms(self):
        self.M=cv2.getPerspectiveTransform(self.src,self.dst) #M will stay the same for all images
        self.M_inv=cv2.getPerspectiveTransform(self.dst,self.src) #M_inv will stay the same for all images
    
    def warp_image(self,image):
        return cv2.warpPerspective(image,self.M,(720,1280),flags=cv2.INTER_LINEAR)
    
    def unwarp_image(self,image):
        return cv2.warpPerspective(image,self.M_inv,(1280,720),flags=cv2.INTER_LINEAR)

In [8]:
#I looked at the image 'Straight Lines 1', and found that near the bottom of the image, the lane width is 705 pixels.
#This could be used to get the height of the camera from the road

#This is simply image distortion such that the source points fit the destination points


st_lines_img=cv2.imread('../test_images/straight_lines1.jpg')

#for pt in img_pts:
#    cv2.circle(st_lines_img,tuple(pt),1,(255,0,0),10)

#cv2.imshow('st_lines',st_lines_img)
#cv2.waitKey(2000)

#ground_pts=np.float32([[0,0],[0,3.6576],[24.384,3.6576],[24.384,0]]) #It seems that the ground points also need to be in pixels

warper=ImageWarper()

dst=warper.warp_image(st_lines_img)
#cv2.imshow('st_lines',dst)
#cv2.waitKey(2000)

binary_images_directory=glob.glob('../output_images/test*.jpg')
canny_images_directory=glob.glob('../output_images/canny/test*.jpg')

binary_images=[cv2.imread(img_name) for img_name in binary_images_directory]
canny_images=[cv2.imread(img_name) for img_name in canny_images_directory]

warped_images_directory='../output_images/warped/'
warped_canny_images_directory='../output_images/warped/canny/'

for image_no,image in enumerate(binary_images):
    dst=warper.warp_image(image)
    
    canny_image=canny_images[image_no]
    dst_canny=warper.warp_image(canny_image)
    #cv2.imshow('img',dst)
    cv2.imwrite(warped_images_directory+'test'+str(image_no+1)+'.jpg',dst)
    cv2.imwrite(warped_canny_images_directory+'test'+str(image_no+1)+'.jpg',dst_canny)


## Detect Lane Pixels in Warped Images

In [8]:
#Detect the peak of the histogram of white pixels in the bottom half of the image
from matplotlib import pyplot as plt

def histogram(image,top=800,bottom=1250):
    #image is assumed to be binary
    window_start=0
    values=np.sum(image[top:bottom][:],axis=0)
    #print(values)
    return np.sum(values,axis=1)/3/1000

def get_peaks(values,tolerance=5):
    peaks=[]
    midpoint=int(len(values)/2)
    
    peaks.append(np.argmax(values[0:midpoint]))
    peaks.append(midpoint+np.argmax(values[midpoint:]))
    
    return peaks #these are pixels


In [52]:
class Box:
    def __init__(self,image,center_x=30,bottom_y=1250,box_height=120,box_width=120,threshold=500):
        self.image=image
        
        self.bottom_y=bottom_y+box_height
        self.center_x=center_x
        self.box_width=box_width
        self.box_height=box_height
        
        self.update_coords(self.center_x)
        self.points=[(self.center_x,bottom_y)]
        self.threshold=threshold #If these many pixels are found, a line exists
        
        self.search_cutoff=500 #Search for only these amount of pixels
        
    #Working Fine
    def get_mean_x(self):
        x_values=0
        n_occurences=0
        n_iterations=0
        
        non_zero_x=np.nonzero(self.image[self.top_y:self.bottom_y,self.left:self.right])[1]
        
        if len(non_zero_x)>self.threshold:
            return self.left+int(np.mean(non_zero_x)),True
        
        return self.center_x,False
    
    def update_coords(self,center_x):
        self.bottom_y-=int(self.box_height)
        self.left=max(int(center_x-(self.box_width/2)),0)
        self.top_y=max(int(self.bottom_y-self.box_height),0)
        self.right=min(self.image.shape[1]-1,int(center_x+self.box_width/2))
        
        self.center_x=center_x
    
    def update_points(self):
        y=self.bottom_y-int(self.box_height/2)
        x,has_line=self.get_mean_x()
        
        if has_line==False:
            x=self.center_x
        else:
            self.points.append((x,y)) #Only consider boxes that have a line inside
        
        #import ipdb
        #ipdb.set_trace()
        self.update_coords(x)
        
    def get_corners(self):
        return [(self.left,self.top_y),(self.right,self.bottom_y)]
    
class LaneDetector:
    def __init__(self,box_width=100,box_height=60,threshold=60,min_interp_points=4):
        
        
        
        self.box_width=box_width
        self.box_height=box_height
                
        self.left_points=[]
        self.right_points=[]
        self.points=[self.left_points,self.right_points]

        self.box_locations=[[],[]]
        self.polynomial_coeffs=[[],[]]
        
        self.n_points_interpolation=0
        self.min_interp_points=min_interp_points
        
        self.threshold=threshold
        
    def set_image(self,image):
        self.image=image
        
        values=histogram(image)
        self.peaks=get_peaks(values)
        
        left_box=Box(image,center_x=self.peaks[0],bottom_y=1250,box_height=self.box_height,box_width=self.box_width,threshold=self.threshold)
        right_box=Box(image,center_x=self.peaks[1],bottom_y=1250,box_height=self.box_height,box_width=self.box_width*3//2)
        self.boxes=[left_box,right_box]
        
    def update_points(self):
        while(self.boxes[0].top_y>0 and self.boxes[1].top_y>0):
            for i,box in enumerate(self.boxes):
                self.box_locations[i].append(box.get_corners())
                ret=box.update_points()
        
        #self.left_points=self.boxes[0].points
        #self.right_points=self.boxes[1].points
        
    def draw_rectangles(self):
        
        for i in range(len(self.box_locations[0])):
            left_corners=self.box_locations[0][i]
            right_corners=self.box_locations[1][i]
            cv2.rectangle(self.image,left_corners[0],left_corners[1],(255,0,0),1)
            cv2.rectangle(self.image,right_corners[0],right_corners[1],(0,0,255),1)   
    
    def fit_points(self):
        self.update_points()
        for i,box in enumerate(self.boxes):
            interp_points=box.points 
            
            if len(interp_points)<self.min_interp_points:
                continue
            
            x=[]
            y=[]
            for point in interp_points:  #O(img_height/box_height)
                x.append(point[0])
                y.append(point[1])
            self.polynomial_coeffs[i]=np.polyfit(y,x,3) #Choosing cubic polynomials 
            
    def evaluate_polynomial_at(self,y,i):
        coeffs=self.polynomial_coeffs[i]
        return np.polyval(coeffs,y)
    
    def constrain(self,value,min_val,max_val):
        value=min(value,max_val)
        value=max(min_val,value)
        return value
    
    def get_lane_points(self):
        points=[]
        for y in range(300,1250,2):
            top_left=[int(self.evaluate_polynomial_at(y,0)),y]
            top_right=[int(self.evaluate_polynomial_at(y,1)),y]
            
            bottom_left=[int(self.evaluate_polynomial_at(y+1,0)),y+1]
            bottom_right=[int(self.evaluate_polynomial_at(y+1,1)),y+1]
            
            #top_left=self.constrain(left_x,0,719)
            #right_x=self.constrain(right_x,0,719)
            
            points.append([top_left,top_right,bottom_right,bottom_left])
            
        return np.int32(points)
    
    def draw_unwarped_lanes(self,frame,warper):
        points=self.get_lane_points()
        blank_image=np.zeros_like(self.image)
        warped_lanes=cv2.fillPoly(blank_image,points,[0,230,0])
        
        unwarped_lanes=warper.unwarp_image(warped_lanes)
        
        return cv2.bitwise_or(unwarped_lanes,frame)
        

In [91]:
warped_images_directory=glob.glob('../output_images/warped/test*.jpg')
warped_images=[cv2.imread(img_name) for img_name in warped_images_directory]

warped_images_binary=[cv2.threshold(image,0,255,type=0)[1] for image in warped_images]
window_width=50

lane_detectors=[]
warper=ImageWarper()
for image in warped_images_binary: 
    lane_detector=LaneDetector(image)
    lane_detector.update_points()
    lane_detector.draw_rectangles()
    lane_detector.fit_points()
    
    #img=lane_detector.draw_unwarped_lanes(frame,warper)
    
    cv2.imshow('image',lane_detector.image)
    cv2.waitKey(2000)
    lane_detectors.append(lane_detector)
    

AttributeError: 'LaneDetector' object has no attribute 'boxes'

## Process the video

In [55]:
cap=cv2.VideoCapture('../project_video.mp4')
warper=ImageWarper()
i=0

lane_detector=LaneDetector()
while(i<4300):
    # Capture frame-by-frame
    cap.set(cv2.CAP_PROP_POS_MSEC,i*50)
    ret, frame = cap.read()
    i+=1
    #print(i)
 
    # Our operations on the frame come here
    binary_image = get_binary_image(frame,cameraMatrix,distCoeffs) #This one's undistorted
    binary_image=cv2.merge([binary_image]*3) #This converts a one channel image into a 3 channel image
    
    warped_image=warper.warp_image(binary_image)
    warped_color_image=warper.warp_image(frame)
    #ret,warped_image=cv2.threshold(warped_image,0,255,type=0)
    
    lane_detector.set_image(warped_image)
    lane_detector.fit_points()
    #lane_detector.draw_rectangles()
    
    # Display the resulting frame
    unwarped_color_image=lane_detector.draw_unwarped_lanes(frame,warper)
    cv2.imshow('binary_frame',lane_detector.image[400:,:,:])
    cv2.imshow('color_frame',unwarped_color_image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()

error: OpenCV(4.2.0) /io/opencv/modules/calib3d/src/undistort.dispatch.cpp:179: error: (-215:Assertion failed) dst.data != src.data in function 'undistort'


In [ ]:
image[0:100,0:100].nonzero()[1]

In [24]:
cv2.imshow('img',frame)

In [96]:
cv2.imshow('short_image',image[0:400,0:400])

/home/laukik/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/laukik/.local/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


nan

In [66]:
i

421